<a href="https://colab.research.google.com/github/IndraLukasTjahaja/Digital_Credit_Risk/blob/main/Latihan_Credit_Scorecard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Latihan
Google Colab ini adalah template yang dapat digunakan untuk membuat PR/Latihan dari training Credit Scorecard Model dengan Python. Template ini terbagi menjadi dua: yang pertama adalah cara cepat menggunakan library scorecardpy dan yang kedua adalah cara manual.

Peserta training diharapkan dapat membuat scorecard dengan menggunakan kedua metode tersebut.

Setiap peserta memiliki dataset yang berbeda-beda. Hal ini dapat diatur dibagian membaca data

# Membaca Data

## Persiapan data dan impor ke Python

Impor data yang akan digunakan untuk membuat model. Untuk peserta training, gunakan dataset yang telah ditunjuk

In [5]:
# library panda untuk menggunakan data frame
import pandas as pd

# Impor data dari excel xls ke Python
# Pembaca dapat langsung menunjuk kepada file excel yang terletak di website pendamping buku ini
# atau mengunduh terlebih dahulu dan menunjuk ke file excel di lokasi hard drive
# contoh: df = pd.read_excel('C:\User\pembaca\downloads\Credit_Scoring_Dataset.xlsx')

# Untuk Chendra. Tugasnya adalah membuat model dengan dataset berikut:
#df = pd.read_excel('https://tjahaja.com/wp-content/uploads/2021/10/german_credit_easy.xlsx')
df = pd.read_excel('https://github.com/IndraLukasTjahaja/Digital_Credit_Risk/blob/main/Dataset/german_credit_easy.xlsx?raw=true')

# Untuk Dea. Tugasnya adalah membuat model dengan dataset berikut:
#df = pd.read_excel('https://tjahaja.com/wp-content/uploads/2021/10/OnClass_CreditScoring_data_v2.xlsx')
df = pd.read_excel('https://github.com/IndraLukasTjahaja/Digital_Credit_Risk/blob/main/Dataset/OnClass_CreditScoring_data_v2.xlsx?raw=true')

# Untuk Imam. Tugasnya adalah membuat model dengan dataset berikut:
#df = pd.read_excel('https://tjahaja.com/wp-content/uploads/2021/10/simulation_credit_kaggle_laotse.xlsx')
df = pd.read_excel('https://github.com/IndraLukasTjahaja/Digital_Credit_Risk/blob/main/Dataset/simulation_credit_kaggle_laotse.xlsx?raw=true')

# Untuk Marsella. Tugasnya adalah membuat model dengan K-S minimum 50
df = pd.read_excel('https://github.com/IndraLukasTjahaja/Digital_Credit_Risk/raw/main/Credit_Scoring_Dataset.xlsx')

# Mohon delete kode read_excel yang bukan tugas kalian

# Struktur dan Tipe Data

In [6]:
df.head(10) # Menampilkan 10 baris teratas dari dataframe df

,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length
0,22,59000,RENT,123.0,PERSONAL,D,35000,16.02,1,0.59,Y,3
1,21,9600,OWN,5.0,EDUCATION,B,1000,11.14,0,0.10,N,2
2,25,9600,MORTGAGE,1.0,MEDICAL,C,5500,12.87,1,0.57,N,3
3,23,65500,RENT,4.0,MEDICAL,C,35000,15.23,1,0.53,N,2
4,24,54400,RENT,8.0,MEDICAL,C,35000,14.27,1,0.55,Y,4
5,21,9900,OWN,2.0,VENTURE,A,2500,7.14,1,0.25,N,2
6,26,77100,RENT,8.0,EDUCATION,B,35000,12.42,1,0.45,N,3
7,24,78956,RENT,5.0,MEDICAL,B,35000,11.11,1,0.44,N,4
8,24,83000,RENT,8.0,PERSONAL,A,35000,8.90,1,0.42,N,2
9,21,10000,OWN,6.0,VENTURE,D,1600,14.74,1,0.16,N,3


In [ ]:
df.info()

# Data Preparation

## Missing data treatment

Asumsi jika ada baris data yang hilang, maka akan kita buang saja

In [ ]:
# Menggunakan fungsi dropna() untuk menghapus semua baris yang memiliki data kosong
df_clean = df.dropna()
df_clean.reset_index(drop=True)

# Data Exploration

Pada bagian ini silahkan melakukan eksplorasi data seperti yang kita lakukan ketika training

## Pembuatan Variabel baru

Jangan lupa untuk membuat variabel baru dan variabel target dalam bentuk angka (jika belum ada)

In [ ]:
import numpy as np

# Untuk dapat melakukan analisa statistik, harus mengubah variabel gagal_bayar dari string menjadi angka (integer)
# Mengubah target variabel gagal_bayar menjadi angka 0 dan 1
conditions = [
             df_clean['gagal_bayar'] == '1_Gagal', 
             df_clean['gagal_bayar'] == '0_Bayar'
             ]

choices     = [
             1,
             0
             ]
df_clean['default'] = np.select(conditions,choices, default = 0)

# Data Modeling cara cepat dengan Scorecardpy

## Instalasi library scorecardpy terlebih dahulu

In [ ]:
# install scorecardpy
!pip install -q scorecardpy
#!pip install git+git://github.com/shichenxie/scorecardpy.git

## Memilih variabel untuk model

In [ ]:
df_model = df_clean[['default','SLIK_Kolektibilitas','Tujuan_Pinjaman']]

## Persiapan Modeling, train dan test dataset, pembuatan binning dan EDA

In [ ]:
y = df_model.loc[:,'default']
X = df_model.loc[:,df_model.columns != 'default']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

# Dalam bentuk dataframe lengkap
df_train = pd.concat([X_train, y_train], axis = 1)
df_test = pd.concat([X_test, y_test], axis = 1)

# Menghapus indeks dari data yang telah dibuang
df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

# Credit Scoring using logistic regression with library scorecardpy
import scorecardpy as scpy

# Melakukan WOE binning
bins = scpy.woebin(df_train, y = 'default')

# Melakukan WOE binning dengan adjustment
breaks_adj = {
}
bins_adj = scpy.woebin(df_train, y="default", breaks_list=breaks_adj)

scpy.woebin_plot(bins_adj)

## Pembuatan WOE secara otomatis

In [ ]:
train = df_train
test = df_test

# converting train and test into woe values
train_woe = scpy.woebin_ply(train, bins_adj)
test_woe = scpy.woebin_ply(test, bins_adj)

y_train = train_woe.loc[:,'default']
X_train = train_woe.loc[:,train_woe.columns != 'default']
y_test = test_woe.loc[:,'default']
X_test = test_woe.loc[:,train_woe.columns != 'default']

## Model dengan logistic regression dan hasil Scorecard

In [ ]:
# Membuat scorecard dengan menggunakan algoritma logistic regression
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train, y_train)

print(lr.coef_)

## Hasil Scorecard

In [ ]:
# predicted probability
train_pred = lr.predict_proba(X_train)[:,1]
test_pred = lr.predict_proba(X_test)[:,1]

# performance ks & roc ------
train_perf = scpy.perf_eva(y_train, train_pred, title = "train")
test_perf = scpy.perf_eva(y_test, test_pred, title = "test")

# score ------
card = scpy.scorecard(bins_adj, lr, X_train.columns, points0=600, odds0=1/20, pdo=50, basepoints_eq0=False)
print(card)
# credit score
train_score = scpy.scorecard_ply(train, card, print_step=0)
test_score = scpy.scorecard_ply(test, card, print_step=0)

# psi
scpy.perf_psi(
  score = {'train':train_score, 'test':test_score},
  label = {'train':y_train, 'test':y_test}
)

